<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/SBShim/LSTM_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install konlpy

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

# 데이터
- 전처리 및 맞춤법 검사가 된 문장

In [2]:
data = pd.read_csv('/content/drive/MyDrive/공부/AIFFELTHON/shared/TUNiB/train_spell_check.csv')

In [6]:
data

,class,conversation,labelling,label_sentence,cleaned_sentence,spell_checked_conversation
0,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,"1,2,1,2,1,2,1,2,1,2",1:길동씨 이번에 이것좀 처리해요\n2:이거 제가 한게 아닌데요\n1:팀에서 내가 ...,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...
1,기타 괴롭힘 대화,야 너 앞니 왜 그렇게 튀어나왔냐?\n 태어날때부터 그랬어 물어보지마.\n 아 그럼...,"1,2,1,2,1,2,1,2,1,1,1,2,1,1",1:야 너 앞니 왜 그렇게 튀어나왔냐?\n2:태어날때부터 그랬어 물어보지마.\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날때부터 그랬어 물어보지마.\n아 그럼 태...,야 너 앞니 왜 그렇게 튀어나왔냐?\n태어날 때부터 그랬어 물어보지 마.\n아 그럼...
2,갈취 대화,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n 미안 나 오늘은 진짜 돈이 없...,"1,2,1,2,1,2,1,2,1,2",1:원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n2:미안 나 오늘은 진짜 돈...,원후야 학원 교재 사야되는데 8만원만 주면 안돼?\n미안 나 오늘은 진짜 돈이 없어...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼?\n미안 나 오늘은 진짜 돈이...
3,갈취 대화,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,"1,2,1,2,1,2,1,2,1,2",1:너 저번에 술먹은 날 기억해?\n2:아니 왜?\n1:야 300만원만 가져와.\n...,너 저번에 술먹은 날 기억해?\n아니 왜?\n야 300만원만 가져와.\n뭔 소리야....,너 저번에 술 먹은 날 기억해?\n아니 왜?\n야 300만 원만 가져와.\n뭔 소리...
4,협박 대화,너 또 내말 무시하냐?\n 이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 ...,"1,1,2,1,1,2,1,2,1,1",1:너 또 내말 무시하냐?\n1:이새끼 널 좆으로 보나본데?\n2:아냐 진짜 시간이...,너 또 내말 무시하냐?\n이새끼 널 좆으로 보나본데?\n아냐 진짜 시간이 없었어 미...,너 또 내말 무시하냐?\n이 새끼 널 좆으로 보나 본데?\n아냐 진짜 시간이 없었어...
...,...,...,...,...,...,...
3908,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...,"1,2,1,2,1,2,1,2,1,2,1",1:준하야 넌 대가리가 왜이렇게 크냐?\n2:내 머리가 뭐.\n1:밥먹으면 대가리만...,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...,준하야 넌 대가리가 왜 이렇게 크냐?\n내 머리가 뭐.\n밥 먹으면 대가리만 크냐 ...
3909,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,"1,2,1,2,1,2,1,2,1,2",1:내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3910,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,"1,1,1,1,1,1,1,1,1,1,2",1:나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n1:왜 그래. ...,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3911,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...,"1,2,1,2,1,2,1,2,1,2",1:누구맘대로 여기서 장사하래?\n2:이게 무슨일입니까?\n1:남의 구역에서 장사하...,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...,누구 맘대로 여기서 장사하래?\n이게 무슨 일입니까?\n남의 구역에서 장사하려면 자...


# 토큰화
- Tensorflow Tokenizer사용
- 단어장 32000개
- 클래스 라벨 one-hot vector화
- train : valid : test = 0.7 : 0.15 : 0.15

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 32000

In [ ]:
sentences = data['spell_checked_conversation']

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(sentences)
word_dic = tokenizer.word_index
#print(word_dic)

sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

padded = pad_sequences(sequences)
print(padded)

In [14]:
len(word_dic)

40198

In [13]:
np.shape(padded)

(3913, 252)

In [15]:
train_data = padded
train_label = data['class']

In [16]:
print(len(train_data), len(train_label))

3913 3913


In [21]:
pd.unique(train_label)

array(['직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '갈취 대화', '협박 대화'], dtype=object)

In [22]:
labels = {'직장 내 괴롭힘 대화':0, '기타 괴롭힘 대화':1, '갈취 대화':2, '협박 대화':3}

In [24]:
train_label = train_label.apply(lambda x: labels[x])

In [36]:
train_label = pd.get_dummies(train_label)

In [37]:
print(train_label)

      0  1  2  3
0     1  0  0  0
1     0  1  0  0
2     0  0  1  0
3     0  0  1  0
4     0  0  0  1
...  .. .. .. ..
3908  0  1  0  0
3909  0  0  1  0
3910  1  0  0  0
3911  0  0  1  0
3912  1  0  0  0

[3913 rows x 4 columns]


In [41]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.3, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

In [42]:
print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

2739 587 587
2739 587 587


# 모델 
- LSTM 사용

In [44]:
word_vector_dim = 32
labels_size = len(labels)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(20))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dense(labels_size, activation='softmax'))


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          1024000   
                                                                 
 lstm_3 (LSTM)               (None, 20)                4240      
                                                                 
 dense_6 (Dense)             (None, 32000)             672000    
                                                                 
 dense_7 (Dense)             (None, 4)                 128004    
                                                                 
Total params: 1,828,244
Trainable params: 1,828,244
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              
epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
11/11 [==============================] - 8s 426ms/step - loss: 1.3771 - accuracy: 0.2979 - val_loss: 1.3443 - val_accuracy: 0.3952
Epoch 2/10
11/11 [==============================] - 4s 420ms/step - loss: 1.1733 - accuracy: 0.4761 - val_loss: 1.0738 - val_accuracy: 0.4412
Epoch 3/10
11/11 [==============================] - 5s 433ms/step - loss: 0.8436 - accuracy: 0.5838 - val_loss: 0.9512 - val_accuracy: 0.5843
Epoch 4/10
11/11 [==============================] - 4s 343ms/step - loss: 0.5599 - accuracy: 0.7722 - val_loss: 0.8094 - val_accuracy: 0.6678
Epoch 5/10
11/11 [==============================] - 4s 347ms/step - loss: 0.2743 - accuracy: 0.9127 - val_loss: 0.8661 - val_accuracy: 0.6848
Epoch 6/10
11/11 [==============================] - 4s 341ms/step - loss: 0.0847 - accuracy: 0.9825 - val_loss: 1.0634 - val_accuracy: 0.7087
Epoch 7/10
11/11 [==============================] - 4s 394ms/step - loss: 0.0224 - accuracy: 0.9971 - val_loss: 1.0831 - val_accuracy: 0.7291
Epoch 

In [46]:
results = model.evaluate(test_X, test_Y, verbose=2)
print(results)

19/19 - 0s - loss: 1.8885 - accuracy: 0.6695 - 444ms/epoch - 23ms/step
[1.8885184526443481, 0.6695059537887573]
